In [159]:
%matplotlib inline

In [160]:
import numpy as np
from skimage.io import imshow, imread, imsave
from skimage.color import rgb2gray
from skimage.exposure import equalize_hist
import pandas as pd

In [161]:
def white_filter_fun(mean):
    return mean >= 0.9

In [162]:
def get_mean(shape, row, col, padding):
    if row + padding > shape.shape[0] or col + padding > shape.shape[1]:
        return None

    return np.mean(shape[row:row + padding,
                         col:col + padding])

In [186]:
def get_max_ratio(img, filter_fun, min_square=25):
    # Problem: includes overlapping boxes
    max_size_found = 0
    max_ratio_found = 0
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            padding = max(min_square, max_size_found) # Minimize size square
            if filter_fun(img[i, j]):
                mean = get_mean(img, i, j, padding)
                if not mean:
                    continue
                while filter_fun(mean):
                    padding += 1
                    mean = get_mean(img, i, j, padding)
                    if not mean:
                        break
                sizes.append(padding)
                if padding > max_size_found:
                    max_size_found = padding
                    img_area = img.shape[0] * img.shape[1]
                    max_ratio_found = ((padding * padding) / float(img_area)) * 100
    return max_ratio_found

In [185]:
def get_sizes(img, filter_fun, min_square=25):
    # Problem: includes overlapping boxes
    sizes = []
    ratios = []
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            padding = min_square # Minimize size square
            if filter_fun(img[i, j]):
                mean = get_mean(img, i, j, padding)
                if not mean:
                    continue
                while filter_fun(mean):
                    padding += 1
                    mean = get_mean(img, i, j, padding)
                    if not mean:
                        break
                sizes.append(padding)
                img_area = img.shape[0] * img.shape[1]
                ratios.append(((padding * padding) / float(img_area)) * 100)
    return sizes, ratios

In [166]:
from transform import load_training
training = load_training()

In [190]:
ratios = []
for i, row in training.iterrows():
    row_patches = []
    row_output = []
    name = row['name']
    fga = row['fga']
    print("Transforming image %s" % name)
    import sys
    sys.stdout.flush()
    for kind in ["DX", "TS"]:
        img = np.load("images-cropped/%s/%s-%s.png" % (kind, name, kind))
        ratio = get_max_ratio(img, white_filter_fun)
        ratios.append(ratio)
        print("Ratio %0.2f" % ratio)

Transforming image 10500
Ratio 0.07
Ratio 0.03
Transforming image 10549
Ratio 0.46
Ratio 0.04
Transforming image 11169
Ratio 0.02
Ratio 0.39
Transforming image 13188
Ratio 0.20
Ratio 0.04
Transforming image 13699
Ratio 0.12
Ratio 0.39
Transforming image 14812
Ratio 0.21
Ratio 0.01
Transforming image 15160
Ratio 0.06
Ratio 0.56
Transforming image 15456
Ratio 0.05


KeyboardInterrupt: 